In [47]:
import requests
import pandas as pd
from tqdm import tqdm
S = requests.Session()

URL = "https://en.wikipedia.org/w/api.php"

COMMON_PARAMS = {
    "action":"query",
    "format":"json",
    "list":"categorymembers",
    "cmlimit": 500
}

PAGES_LIST = []

PAGES_DICT = {
}

FIRST_RUN = True

In [38]:
CATEGORIES_LIST = pd.read_csv("scraped_categories.csv")['0'].tolist()
CATEGORIES_LIST.append("Category:Archaeological artifacts")

In [39]:
def scrape_page_text(title):
    PARAMS = {
        "action":"query",
        "format":"json",
        'prop': 'extracts',
        'explaintext': True,
        'titles': title
    }
    R = S.get(url=URL, params=PARAMS)
    article = R.json()
    pages = article['query']['pages']
    page_id = next(iter(pages))
    text = pages[page_id]['extract']
    return text

In [40]:
scrape_page_text("Ain Sakhri Lovers")

'The Ain Sakhri Lovers figurine is a sculpture that was found in one of the Ain Sakhri caves near Bethlehem. The sculpture is considered to be 11,000 years old and to be the oldest known representation of two people engaged in sexual intercourse.\n\n\n== Discovery ==\nThe sculpture was identified in 1933 by René Neuville, a French consul in Jerusalem and prehistorian, when looking through random finds obtained by the French Fathers at Bethlehem. He found the stone whilst visiting a small museum with Abbé Breuil. Neuville immediately identified it as important and was able to get an introduction to the Bedouin who had made the finds at Wadi Khareitoun. He was led to a location within the Ain Sakhri caves and it is from these caves that the sculpture gets its name. Excavations of the caves revealed that the cave had been used domestically thousands of years ago and the finds were Natufian. For this reason it is thought that the figurine was used domestically and had not been left there a

In [41]:
def scrape_page_wikidata(title):
    PARAMS = {
        "prop": "pageprops",
        "ppprop": "wikibase_item",
        "format": "json",
        "action": "query",
        "titles": title
    }
    R = S.get(url=URL, params=PARAMS)
    metadata = R.json()
    metadata = metadata['query']['pages']
    page_id = next(iter(metadata))
    entity = metadata[page_id]["pageprops"]["wikibase_item"]
    return entity

In [42]:
scrape_page_wikidata("William the Faience Hippopotamus")

'Q137836'

In [43]:
def rec_scrape_pages(category_name, cmcontinue = False):
    PARAMS = COMMON_PARAMS
    PARAMS["cmtype"] = "page"
    PARAMS["cmtitle"] = category_name
    R = S.get(url=URL, params=PARAMS)
    articles = R.json()
    if articles.get('continue', False):
        cmcontinue = articles['continue']['cmcontinue']
    payload = articles['query']['categorymembers']
    for page in payload:
        title = page['title']
        if page in PAGES_LIST:
            continue
        else:
            try:
                text = scrape_page_text(title)
                entity = scrape_page_wikidata(title)
                PAGES_DICT[title] = {
                    "category": category_name,
                    "title": title,
                    "text": text,
                    "entity": entity
                }
            except Exception as e:
                print("An exception occurred at page ", title)
                print(e)
    if cmcontinue:
        rec_scrape_pages(category_name, cmcontinue)

In [48]:
for category in tqdm(CATEGORIES_LIST):
    global FIRST_RUN
    global PAGES_DICT
    global PAGES_LIST
    rec_scrape_pages(category)
    PAGES_LIST.extend(PAGES_DICT.keys())
    if FIRST_RUN:
        df = pd.DataFrame.from_dict(PAGES_DICT, orient="index").reset_index(drop=True)
        FIRST_RUN = False
    else:
        old = pd.read_csv("scraped.csv")
        new = pd.DataFrame.from_dict(PAGES_DICT, orient="index").reset_index(drop=True)
        df = pd.concat([old, new])
    df = df[df.columns.drop(list(df.filter(regex='Unnamed: 0')))]
    df.to_csv("scraped.csv")

  0%|          | 1/628 [00:43<7:29:37, 43.03s/it]/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]
  0%|          | 2/628 [00:55<5:53:20, 33.87s/it]

An exception occurred at page  Template:Greek vase painting
'pageprops'


  0%|          | 3/628 [01:05<4:38:21, 26.72s/it]

An exception occurred at page  Template:Greek amphorae
'pageprops'


  1%|▏         | 8/628 [02:34<4:26:34, 25.80s/it]

...

  5%|▍         | 29/628 [05:58<1:56:52, 11.71s/it]

An exception occurred at page  A Goddess Travels to Egypt
'pageprops'
An exception occurred at page  Love and Gold
'pageprops'
An exception occurred at page  The Missing Gold Statues Again
'pageprops'
An exception occurred at page  Tushratta to Queen Mother Tiye, and missing gold statues
'pageprops'


  5%|▍         | 30/628 [06:06<1:45:36, 10.60s/it]

An exception occurred at page  A Hymn to the Pharaoh
'pageprops'
An exception occurred at page  A Plea for Help
'pageprops'
An exception occurred at page  A Reckoning Demanded
'pageprops'
An exception occurred at page  A Royal Order for Glass
'pageprops'
An exception occurred at page  A Throne Granted, Not Inherited
'pageprops'
An exception occurred at page  A Very Serious Crime
'pageprops'
An exception occurred at page  Abimilku letter 149: Neither water nor wood
'pageprops'


KeyboardInterrupt: 